In [ ]:
import pandas as pd
import numpy as np
import json
import os
pd.set_option('display.max_columns', 256)
pd.set_option('display.max_rows', 512)
import overpy
from geopy import distance
from pyproj import Transformer
import h3
import math
from sklearn.metrics import pairwise
import requests
import shapely
import geopandas as gpd
import regex
from keplergl import KeplerGl


## Часть I. О вычислениях расстояний

### Как правило географические координаты представлены в градусах как единицах измерения.

### При этом важно понимать, к какой CRS (системе координат) относятся данные кооординаты.

### Системы координат могут быть глобальными (покрывать всю поверхность планеты) либо локальными (покрывать ее определенную часть)

### Чаще всего этой системой координнат является [WGS-84](https://epsg.io/4326) (глобальная система координат в градусах) - используется в подавляющем количестве картографических сервисов, GPS-приборов и проч.


In [ ]:
# пример - нулевой километр
zero_km = (55.755797, 37.617729)

# пример - аэропорт Шереметьево
sherem = (55.981359, 37.413861)

### Считать расстояние между координатами напрямую как правило нельзя - если на входе градусы, расстояние также будет в градусах.
    
### Домножить на "среднюю" длину 1 градуса может давать серьезные искажения (чем больше расстояние - тем выше искажения)

### WGS-84 имеет в основе определенную модель эллипсоида - сферы, "сплюснутой" вдоль одной из осей (полярный радиус чуть меньше экваториального).

### Разные модели геоидов отличаются разным отношением полярного и экваториального радиусов - параметр flattening (сплюснутость)

![ellipsoids](pics/ellipsoids.png)


### Для вычисления расстояний на эллипсоиде проще всего использовать библиотеку geopy


In [ ]:
meters_wgs84 = distance.geodesic(zero_km, sherem, ellipsoid='WGS-84').m
print(f'WGS-84 distance (meters): {meters_wgs84:.2f}')

### Однако куда проще "округлить" эллипсоид до сферы и считать расстояние по сфере [great-circle disatnce](https://en.wikipedia.org/wiki/Great-circle_distance)


In [ ]:
meters_great_circle = distance.great_circle(zero_km, sherem).m
print(f'great-circle distance (meters): {meters_great_circle:.2f}')

### Что быстрее?

In [ ]:
%%timeit
distance.geodesic(zero_km, sherem).m

In [ ]:
%%timeit
distance.great_circle(zero_km, sherem).m

### Что если посчитать расстояние в градусах, а потом перевести в метры исходя из длины экватора/меридиана?

In [ ]:
# длина меридиана в км
meridian_length_km = 20003.93

# длина экватора в км
equator_length_km = 40075

# евклидово расстояние в градусах:
euclidean = pairwise.euclidean_distances(
    np.asarray([zero_km]),
    np.asarray([sherem])
)[0][0]

# делим на 180 (широта от -90 до 90)
print(f'equclidian distance (meters) by meridian: {euclidean*meridian_length_km*1000/180.0:.2f}')

# делим на 360 (долгота от -180 до 180)
print(f'equclidian distance (meters) by equator: {euclidean*equator_length_km*1000/360.0:.2f}')


### P. S. эллипсоиды, которые используются в некоторых глобальных системах координат рассчитаны на то, чтобы более-менее точно описывать форму поверхности планеты везде

### Для самых точных геодезических вычислений подобраны "локальные" эллипоиды, наиболее точно описывающие конкретный участок поверхности


![local_ellipsoid](pics/image17.png)

### Важно!

### В sclearn есть встроенная метрика расстояния на сфере - haversine (тот самый great-circle disntace)

### В некоторых задачах (например, density-based кластеризации) полезно использовать ее

### В общем случае проще сконвертировать градусы в радианы, а параметр расстояния указывать не в метрах, а нормировать на длину радиуса



In [ ]:
# Чтобы посчитать haversine расстояние в метрических единицах, нужно:

#1. Перевести координаты из градусов в радианы

zero_km_rad = tuple(math.radians(val) for val in zero_km)
sherem_rad = tuple(math.radians(val) for val in sherem)

#2. Полученное (безразмерное) расстояние умножить на радиус сферы - средний радиус Земли

haversine = pairwise.haversine_distances(
    np.asarray([zero_km_rad]),
    np.asarray([sherem_rad])
)[0][0]

print(f'haversine distance: {haversine:.6f}')

#средний радиус Земли в км
R = 6371

print(f'haversine distance (meters): {haversine*R*1000:.2f}')


## Что делать если не точки, а геометрии (множества точек) - полигоны, линии и проч.?

### Зачастую не нужно считать расстояния между слишком удаленными геометриями.

### В этом случае есть удобные локальные системы координат - [UTM](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system)

### Набор UTM проекций разбивает поверхность Земли на отдельные зоны - от экватора до 84 параллели (на север и юг) и шириной по 6 градусов долготы 

### В таких системах координат расстояния можно считать в метрах (так как сами координаты тоже в метрах)

### Для полюсов существуют отдельные [UPS](https://en.wikipedia.org/wiki/Universal_polar_stereographic_coordinate_system) проеции

### Для каждой такой зоны есть отдельная проекция на плоскость с метрическими координатами. При проекции искажения не превышают 0.1% в пределах такой зоны

### Код зоны можно вычислить по координатам точки


### Существует также и глобальная система координат в метрах - https://epsg.io/3857, но она сильно искажает расстояния



![UTM](pics/image20.png)

In [ ]:
# Пример 
def get_utm_code(lat: float, lon:float):
    utm_band = int((lon+180) / 6) % 60 + 1
    utm_band = f'0{utm_band}'[-2:]
    utm  = ('EPSG:326' if lat > 0 else 'EPSG:327') + utm_band
    return utm

print(f'https://epsg.io/{get_utm_code(zero_km[0], zero_km[1])[5:]}')

In [ ]:
#исходная проеция - WGS-84
source_crs = 'EPSG:4326'
#целевая проекция - UTM зона
target_crs = get_utm_code(zero_km[0], zero_km[1])
transformer = Transformer.from_crs(source_crs, target_crs)

#перевод координат в UTM
zero_km_utm = transformer.transform(zero_km[0], zero_km[1])
print(f'wgs-84 coords for zero_km: {zero_km}')
print(f'utm coords for zero_km: {zero_km_utm}\n')
sherem_utm = transformer.transform(sherem[0], sherem[1])
print(f'wgs-84 coords for sherem: {sherem}')
print(f'utm coords for sherem: {sherem_utm}\n')

# евклидово расстояние в проекции:
euclidean_utm = pairwise.euclidean_distances(
    np.asarray([zero_km_utm]),
    np.asarray([sherem_utm])
)[0][0]

print(f'euclidian distance in UTM projection (meters): {euclidean_utm:.2f}')


### Для работы с геометриями есть удобная библиотеки - shapely и geopandas


[Москва](https://www.openstreetmap.org/relation/2555133)

In [ ]:
moscow = requests.get('https://polygons.openstreetmap.fr/get_wkt.py?id=2555133&params=0').text
print(moscow)
moscow = shapely.wkt.loads(moscow[10:])

In [ ]:
moscow

In [ ]:
# площадь Москвы ( в квадратных градусах:) )
print(f'area (wrong!): {moscow.area}')

In [ ]:
print(str(type(moscow)))
print('\n')
print(str(moscow)[:500])

[Коломна](https://www.openstreetmap.org/relation/1703080#map=7/54.617/39.902)

In [ ]:
kolomna = requests.get('https://polygons.openstreetmap.fr/get_wkt.py?id=1703080&params=0').text
kolomna = shapely.wkt.loads(kolomna[10:])

In [ ]:
kolomna

[Подольск](https://www.openstreetmap.org/relation/1697322#map=7/54.617/39.902)

In [ ]:
podolsk = requests.get('https://polygons.openstreetmap.fr/get_wkt.py?id=1697322&params=0').text
podolsk = shapely.wkt.loads(podolsk[10:])

In [ ]:
# соберем датафрейм
df = pd.DataFrame({
    'city': ['moscow', 'kolomna', 'podolsk'],
    'wkt': [str(city) for city in [moscow, kolomna, podolsk]]
})
df.head()

In [ ]:
# конвертнем в геопандас
df['wkt'] = gpd.GeoSeries.from_wkt(df['wkt'])
gdf = gpd.GeoDataFrame(df, geometry='wkt')


In [ ]:
gdf.head()

In [ ]:
gdf.iloc[0].wkt

In [ ]:
# в геопандас можно задать CRS (систему координат) и перевести в другую CRS в случае необходимости
print(gdf.crs)

gdf = gdf.set_crs('EPSG:4326')
print(gdf.crs)


In [ ]:
# найдем центр Москвы и конвертнем в его CRS
moscow_center = list(moscow.centroid.coords)[0]
moscow_center_crs = get_utm_code(moscow_center[0], moscow_center[1])
print(moscow_center_crs)
gdf = gdf.to_crs(moscow_center_crs)
print(gdf.crs)


In [ ]:
gdf.iloc[0].wkt

In [ ]:
# Теперь посчитаем площадь
gdf['area'] = gdf.wkt.apply(lambda t: t.area/1000**2)
gdf.head()

In [ ]:
# Расстояние от Москвы до Коломны 
dist = gdf[gdf.city=='moscow'].wkt.iloc[0].distance(
    gdf[gdf.city=='kolomna'].wkt.iloc[0]
)
print(f'distance from Moscow to Kolomna (km): {dist/1000:.2f}')


### Для более серьезных вычислений лучше использовать отдельные инструменты:

### - [PostGIS](https://postgis.net/workshops/postgis-intro/)

### - [Apache Sedona](https://sedona.apache.org)

### - [GeoMesa](https://www.geomesa.org)

## Часть II. Получение данных

### Самый простой источник в плане получения данных - OSM (OpenStreetMap)

### Для получения большого объема данных из OSM по сути есть 2 базовых пути:

### 1. Общедоступные API (отнсительно просто запустить, но могут быть сложности в получении действительно большого объема данных)

### пример - OverPass ([wiki](https://wiki.openstreetmap.org/wiki/Overpass_API), [examples](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example))

### 2. Поднять свой PostGIS сервер и загрузить данные из дампа OSM данных:

### есть куча туториалов ([пример](https://www.linuxbabe.com/linux-server/openstreetmap-tile-server-ubuntu-16-04))

### и сервисов где можно скачать дамп OSM как полный (~40 Gb) так и нарезанный отдельно по странам/регионам (примеры - [Россия](http://download.geofabrik.de/russia.html), [отдельные регионы РФ](http://osmosis.svimik.com/latest/))

### Будем использовать overpy - python либу для Overpass API

In [ ]:
API  = overpy.Overpass()

### достанем список (крупных) городов [по тэгу place=city](https://wiki.openstreetmap.org/wiki/Tag:place%3Dcity)

### опция 1: использовать API через либу overpy

In [ ]:
# достанем nodes и их координаты

result = API.query("""
area[name="Россия"][admin_level=2]->.boundaryarea;
(
nwr(area.boundaryarea)[place="city"];
);
out meta;
""")

In [ ]:
cities_points = \
pd.DataFrame(
    [
        (
            node.tags.get('name'),
            int(node.tags.get('population')),
            float(node.lat),
            float(node.lon),
        )
        for node in result.nodes
    ],
    columns = ['name', 'population', 'lat', 'lon']
).sort_values('population', ascending=False)

cities_points.head(15)

### опиця 2 - использовать через requests

In [ ]:
# достанем relations и их полигоны

url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

query = """[out:json];
area[name="Россия"][admin_level=2]->.boundaryarea;
(
relation(area.boundaryarea)['place' = 'city'];
);
convert item ::=::,::geom=geom(),_osm_type=type();
out geom;"""

response = requests.get(url, params={'data': query})
data = response.json()


In [ ]:
json_data = [
    {
        'id': element['id'],
        'name': element['tags'].get('name'),
        'population': element['tags'].get('population'),
        'geometry': shapely.geometry.MultiPolygon(
            shapely.ops.polygonize(shapely.geometry.shape(element['geometry']))
        ),
    }
    for element in data['elements']
]

cities_polygons = gpd.GeoDataFrame(json_data)

cities_polygons['population'] = cities_polygons['population'].apply(
    lambda t: int(t.replace(' ', '')) if t else None
)

cities_polygons.head(15)

### P. S.

### Но есть и самые неочевидные источники, где можно достать данные

### Иногда данные можно распарсить из страницы или API, к которому обращается страница

### Для таких кейсов полезно смотреть на вкладку сетевой активности в браузере

In [ ]:
# главная страница 2gis

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6 Safari/605.1.15'
}
response_2gis = requests.get('https://2gis.ru/', headers=headers)
print(response_2gis.status_code)
text_2gis = requests.get('https://2gis.ru/', headers=headers).text


In [ ]:
# найдем json с городами 
pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
all_jsons = pattern.findall(text_2gis)


In [ ]:
# найдем нужный json
data_2gis = json.loads(
    [
        el for el in all_jsons if 'geometry' in el
    ][0]
)

settlements = data_2gis.get('data').get('settlements').get('default').get('data')
city = data_2gis['data']['entity']['profile']['4504222397630173']


In [ ]:
#вот тут лежат все возможные города
settlements

In [ ]:
#вот Москва (есть ID и алиас)
settlements['4504222397630173']

In [ ]:
#а тут есть данные по конкретному городу
city

#### теперь можно пробежаться по url-ам вида f'https://2gis.ru/{alias}' и собрать json-ы по каждому городу

## Часть III Визуализация и индексы

### По части визуализации есть разные инструменты. Среди рекоммендуемых следующие:

### [kepler.gl](https://kepler.gl) - python-библиотека, удобное отрисовывание в jupyter ноутбкуке, поддержка многих форматов данных

### [QGIS](https://www.qgis.org/ru/site/about/index.html) - отдельное приложение. Может отрисовывать несколько больший объем данных. Есть возможность редактирования векторых слоев, запуска python-скриптов



### прочитаем датафрейм с недвижимостью Москвы

In [ ]:
!unzip data/realty_data.csv.zip -d data/

In [ ]:
!unzip data/realty_data_validation.csv.zip -d data/

In [ ]:
realty_data = pd.read_csv('data/realty_data.csv')
realty_data.head(5)

### добавим индекс

### в качестве индекса будем использовать [гексагональную сетку Uber H3](https://h3geo.org/docs/highlights/indexing)

### индекс чаще всего представляет собой шестиугольник определенного разрешения (длины ребра/площади). Всего существует 16 различных разрешений - [см. таблицу](https://h3geo.org/docs/core-library/restable/)


In [ ]:
# добавим индекс
RES = 8
print(f'edge length in meters for resolution {RES} is {h3.edge_length(RES)*1000:.2f}')


In [ ]:
realty_data['price_m2'] = realty_data.apply(
    lambda t: round(t.price/t.total_square)/1000, axis=1
)

realty_data['hex_id'] = realty_data.apply(
    lambda t: h3.geo_to_h3(t.lat, t.lon, RES), axis=1
)

realty_data.head()

In [ ]:
#средняя цена квадрата по индексу
hex_df = realty_data.groupby('hex_id')[['price', 'total_square']].sum().reset_index()
hex_df['price_m2'] = hex_df.apply(
    lambda t: round(t.price/t.total_square)/1000.0, axis=1
)

hex_df.head()


In [ ]:
#теперь сгладим цену взяв среднее по соседним индексам (в радиусе 2 индексов)

#соберем в словарь
hex_dict = dict(
    (hex_id, {'price': price, 'total_square': total_square})
    for hex_id,  (price, total_square) in zip(hex_df.hex_id, zip(hex_df.price, hex_df.total_square))
)


In [ ]:
RADIUS = 2

hex_df['r2_price'] = hex_df.hex_id.apply(
    lambda h: sum(hex_dict.get(n_hex, {}).get('price', 0) for n_hex in h3.k_ring(h, RADIUS))
)

hex_df['r2_total_square'] = hex_df.hex_id.apply(
    lambda h: sum(hex_dict.get(n_hex, {}).get('total_square', 0) for n_hex in h3.k_ring(h, RADIUS))
)

hex_df['r2_price_m2'] = hex_df.apply(
    lambda t: round(t.r2_price/t.r2_total_square)/1000.0, axis=1
)

hex_df.tail()

In [ ]:
realty_map = KeplerGl()
realty_map.add_data(realty_data.drop(['description', 'hex_id'], axis=1), name='realty')
realty_map.add_data(hex_df, name='hex_avg_price')


In [ ]:
realty_map

In [ ]:
realty_map.save_to_html(file_name='realty.html')

### удалим в конце файл из-за большого размера

In [ ]:
os.remove('realty.html')